In [27]:
# Support functions
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform

# Fit models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Scoring functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [28]:
import base64
import hashlib
from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes

__key__ = hashlib.sha256(b'16-character key').digest()

def encrypt(raw):
    BS = AES.block_size
    pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS)

    raw = base64.b64encode(pad(raw).encode('utf8'))
#     iv = get_random_bytes(AES.block_size)
#     print(iv)
    iv = b'f5FI\t\xcau\x16gq\x9f_\xb7\xc3\xc3t'
    cipher = AES.new(key= __key__, mode= AES.MODE_CFB,iv= iv)
    return base64.b64encode(iv + cipher.encrypt(raw))

def decrypt(enc):
    unpad = lambda s: s[:-ord(s[-1:])]

    enc = base64.b64decode(enc)
    iv = enc[:AES.block_size]
    cipher = AES.new(__key__, AES.MODE_CFB, iv)
    return unpad(base64.b64decode(cipher.decrypt(enc[AES.block_size:])).decode('utf8'))


In [29]:
import pandas as pd

In [30]:
df = pd.read_csv("Churn_Modelling.csv")

In [31]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [32]:
for i in range(len(df['CustomerId'])):
    df['CustomerId'][i] = encrypt(str(df['CustomerId'][i]))
    
for i in range(len(df['CreditScore'])):
    df['CreditScore'][i] = encrypt(str(df['CreditScore'][i]))
    
for i in range(len(df['Tenure'])):
    df['Tenure'][i] = encrypt(str(df['Tenure'][i]))

<ipython-input-32-050795f36386>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CustomerId'][i] = encrypt(str(df['CustomerId'][i]))
C:\Users\dell\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-32-050795f36386>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CreditScore'][i] = encrypt(str(df['CreditScore'][i

In [33]:
## REQUIRED LIBRARIES
# For data wrangling 
import numpy as np
import pandas as pd

# For visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [34]:
# Split Train, test data
df_train = df.sample(frac=0.8,random_state=200)
df_test = df.drop(df_train.index)
print(len(df_train))
print(len(df_test))

8000
2000


In [ ]:
def byte2int(row):
    return int.from_bytes(row, byteorder='little')
for i in df_train.index:
    df_train['CreditScore'][i] = byte2int(df_train['CreditScore'][i])
    
    
for i in df_train.index:
    df_train['Tenure'][i] = byte2int(df_train['Tenure'][i])


<ipython-input-35-5e07bfe76488>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['CreditScore'][i] = byte2int(df_train['CreditScore'][i])
<ipython-input-35-5e07bfe76488>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Tenure'][i] = byte2int(df_train['Tenure'][i])


In [ ]:
# CS
# age
# tenure
# balance
# no of prod
# esti sal
# cred?
# genfer?
# active mem

In [ ]:
# Arrange columns by data type for easier manipulation
continuous_vars = ['CreditScore',  'Age', 'Tenure', 'Balance','NumOfProducts', 'EstimatedSalary']
cat_vars = ['HasCrCard', 'IsActiveMember','Gender']
df_train = df_train[['Exited'] + continuous_vars + cat_vars]


# df_train.loc[df_train.HasCrCard == 0, 'HasCrCard'] = -1
# df_train.loc[df_train.IsActiveMember == 0, 'IsActiveMember'] = -1

# One hot encode the categorical variables
lst = ['Gender']
# remove = list()
# for i in lst:
#     if (df_train[i].dtype == np.str or df_train[i].dtype == np.object):
#         for j in df_train[i].unique():
#             df_train[i+'_'+j] = np.where(df_train[i] == j,1,-1)
#         remove.append(i)
# df_train = df_train.drop(remove, axis=1)
df_train.head()

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train.loc[:, df_train.columns != 'Exited'],df_train.Exited, test_size = 0.3, random_state = 0)

In [21]:
X_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,HasCrCard,IsActiveMember,Gender_Female,Gender_Male
3882,1738737437789005151539054136451294923882475372...,22,1738737351209833734286625467444490798382037501...,0.00,2,66393.89,1,1,1,-1
1024,1738720915593023773395633731180690884016314770...,25,1738730468447623122025912215773357411190383058...,108691.95,1,63030.97,1,-1,1,-1
283,1738737358270427916698564317649226037274359884...,42,1738713968728647673713442992023755337741900804...,118274.71,1,25885.72,1,1,-1,1
6120,1738720920696659671831347415737705933881442570...,40,1738713968728647673713442992023755337741900804...,0.00,1,141359.11,1,1,-1,1
3281,1738721001768713274719807417750168771839089651...,32,1738720887061892845020098355802042553041505048...,108698.96,2,161069.73,1,1,1,-1


In [22]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))
  
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

X_train_res['CreditScore'] = np.nan_to_num(np.array(X_train_res['CreditScore']).astype(np.float32))
X_train_res['Tenure'] = np.nan_to_num(np.array(X_train_res['Tenure']).astype(np.float32))
X_train_res.head()

X_test['CreditScore'] = np.nan_to_num(np.array(X_test['CreditScore']).astype(np.float32))
X_test['Tenure'] = np.nan_to_num(np.array(X_test['Tenure']).astype(np.float32))
# X_test.head()

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100) 
clf.fit(X_train_res, y_train_res)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print("RF \n", clf.score(X_test, y_test))


from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train_res, y_train_res)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

print("XG \n", model.score(X_test, y_test))

Before OverSampling, counts of label '1': 1140
Before OverSampling, counts of label '0': 4460 

After OverSampling, the shape of train_X: (8920, 10)
After OverSampling, the shape of train_y: (8920,) 

After OverSampling, counts of label '1': 4460
After OverSampling, counts of label '0': 4460
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      1893
           1       0.59      0.54      0.56       507

    accuracy                           0.82      2400
   macro avg       0.73      0.72      0.72      2400
weighted avg       0.82      0.82      0.82      2400

RF 
 0.8220833333333334
              precision    recall  f1-score   support

           0       0.88      0.91      0.89      1893
           1       0.61      0.53      0.57       507

    accuracy                           0.83      2400
   macro avg       0.75      0.72      0.73      2400
weighted avg       0.82      0.83      0.82      2400

XG 
 0.8295833333333333


In [23]:
import pickle
pickle.dump(model, open("float64model.pkl", 'wb'))


In [24]:
X_train_res.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,HasCrCard,IsActiveMember,Gender_Female,Gender_Male
0,3.402823e+38,22,3.402823e+38,0.00,2,66393.89,1,1,1,-1
1,3.402823e+38,25,3.402823e+38,108691.95,1,63030.97,1,-1,1,-1
2,3.402823e+38,42,3.402823e+38,118274.71,1,25885.72,1,1,-1,1
3,3.402823e+38,40,3.402823e+38,0.00,1,141359.11,1,1,-1,1
4,3.402823e+38,32,3.402823e+38,108698.96,2,161069.73,1,1,1,-1


In [26]:
X_train_res['NumOfProducts'].value_counts()

1    5581
2    3067
3     237
4      35
Name: NumOfProducts, dtype: int64

In [18]:
val = X_test.iloc[1:2]
print(val.values)

[[ 3.40282347e+38  5.00000000e+01  3.40282347e+38  8.83798100e+04
   3.00000000e+00  6.41572400e+04 -1.00000000e+00  1.00000000e+00
   1.00000000e+00 -1.00000000e+00]]


In [19]:
# CS
# age
# tenure
# balance
# no of prod
# esti sal
# cred?
# genfer?
# active mem

In [1]:
len([ 3.40282347e+38  ,5.00000000e+01,  3.40282347e+38 , 8.83798100e+04,   3.00000000e+00,  6.41572400e+04, -1.00000000e+00 , 1.00000000e+00 ,1.00000000e+00, -1.00000000e+00])

10

In [28]:
model.predict(np.array([[ 3.40282347e+38  ,5.00000000e+01,  3.40282347e+38 , 8.83798100e+04,   3.00000000e+00,  6.41572400e+04, -1.00000000e+00 , 1.00000000e+00 ,1.00000000e+00, -1.00000000e+00]]))

array([1])

In [31]:
with open("float64model.pkl", "rb") as f:
    rawdata = f.read()

mm = pickle.loads(rawdata)

In [32]:
mm.predict(val)

array([1])

In [28]:

# print("Before Undersampling, counts of label '1': {}".format(sum(y_train == 1)))
# print("Before Undersampling, counts of label '0': {} \n".format(sum(y_train == 0)))
  
# # apply near miss
# from imblearn.under_sampling import NearMiss
# nr = NearMiss()
  
# X_train_miss, y_train_miss = nr.fit_resample(X_train, y_train.ravel())
  
# print('After Undersampling, the shape of train_X: {}'.format(X_train_miss.shape))
# print('After Undersampling, the shape of train_y: {} \n'.format(y_train_miss.shape))
  
# print("After Undersampling, counts of label '1': {}".format(sum(y_train_miss == 1)))
# print("After Undersampling, counts of label '0': {}".format(sum(y_train_miss == 0)))

# X_train_miss['CreditScore'] = np.nan_to_num(np.array(X_train_miss['CreditScore']).astype(np.float32))
# X_train_miss['Tenure'] = np.nan_to_num(np.array(X_train_miss['Tenure']).astype(np.float32))
# # X_train_miss.head()

# X_test['CreditScore'] = np.nan_to_num(np.array(X_test['CreditScore']).astype(np.float32))
# X_test['Tenure'] = np.nan_to_num(np.array(X_test['Tenure']).astype(np.float32))
# # X_test.head()

In [26]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators = 100) 

# clf.fit(np.nan_to_num(np.array(X_train_miss).astype(np.float32)), np.nan_to_num(np.array(y_train_miss).astype(np.float32)))
# X_test = np.nan_to_num(np.array(X_test).astype(np.float32))

# y_pred = clf.predict(X_test)
# print(classification_report(np.nan_to_num(np.array(y_test).astype(np.float32)), y_pred))

# clf.score(X_test, y_test)

In [46]:
import phe
from phe import paillier
import json

In [52]:
# import numpy as np
# # print(np.array(X_train_miss).astype(np.float32).dtype)
# arr = np.array(X_train_miss).astype(np.float64)
# print(arr.dtype)
# print(arr)
# np.nan_to_num(arr)

In [47]:
pub_key,priv_key = paillier.generate_paillier_keypair()

In [49]:
pub_key.encrypt(5).ciphertext()


1468226413192906536615396394199370406653272674007199393145773860993777426683380402999389059412123286075349957065115397007471165162410647066228330000426066750020673784277650299665245930754474430509399847750380543492121005765972251062810658719430958183818399772875247504030933171825457915710572881234374202123665349001819997442621176206601326641635495288287897891645846248843853125066653113531067961691526323371209526103404607517428711229026214916991179311488953596705716348777053723833562555454918881323288336071344401154351326500207313963730744574995569317551550701027528206698562350464932887028957603463473783599819166070372249648129685079453997278593961065700793727611545753377108270956188601154161435605255353896252286225301489028154430446018064931212921889489193059008514014712985848533259216367384963635749003163603779020626681361223592915662786425919586183527840389705286831820005372248431324778522851980182227141879327754569807696699117693664939043283028243334864232285678132785196398769545312